<a href="https://colab.research.google.com/github/nuccigi/desafio_hackathon/blob/main/primeira_entrega.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import zipfile
import os
import pandas as pd

In [1]:
caminho_zip = "/content/drive/MyDrive/hackathon_2025_templates.zip"

# Pasta de destino
destino = "/content/drive/MyDrive/arquivos_extraidos"

# Cria a pasta se não existir
os.makedirs(destino, exist_ok=True)

# Extrai o conteúdo
with zipfile.ZipFile(caminho_zip, 'r') as zip_ref:
    zip_ref.extractall(destino)


NameError: name 'os' is not defined

In [2]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [5]:
# Caminho de um arquivo Parquet
dados1 = "/content/drive/MyDrive/arquivos_extraidos/hackathon_2025_templates/part-00000-tid-2779033056155408584-f6316110-4c9a-4061-ae48-69b77c7c8c36-4-1-c000.snappy.parquet"
dados2 = "/content/drive/MyDrive/arquivos_extraidos/hackathon_2025_templates/part-00000-tid-5196563791502273604-c90d3a24-52f2-4955-b4ec-fb143aae74d8-4-1-c000.snappy.parquet"
dados3 = "/content/drive/MyDrive/arquivos_extraidos/hackathon_2025_templates/part-00000-tid-7173294866425216458-eae53fbf-d19e-4130-ba74-78f96b9675f1-4-1-c000.snappy.parquet"

df1 = pd.read_parquet(dados1)
df2 = pd.read_parquet(dados2)
df3 = pd.read_parquet(dados3)


In [6]:
print(df1.head())

                   pdv      premise categoria_pdv  zipcode
0  2204965430669363375   On Premise  Mexican Rest    30741
1  5211957289528622910   On Premise   Hotel/Motel    80011
2  9024493554530757353  Off Premise   Convenience    80751
3  8659197371382902429   On Premise    Restaurant    80439
4  1400854873763881130   On Premise    Restaurant    30093


In [7]:
print(df2.head())

     internal_store_id  internal_product_id distributor_id transaction_date  \
0  7384367747233276219   328903483604537190              9       2022-07-13   
1  3536908514005606262  5418855670645487653              5       2022-03-21   
2  3138231730993449825  1087005562675741887              6       2022-09-06   
3  3681167389484217654  1401422983880045188              5       2022-09-11   
4  7762413312337359369  6614994347738381720              4       2022-02-18   

  reference_date  quantity  gross_value    net_value  gross_profit  \
0     2022-07-01       1.0    38.125000    37.890625     10.042625   
1     2022-03-01       6.0   107.250000   106.440002     24.732002   
2     2022-09-01       3.0    56.625000    56.220001     14.124002   
3     2022-09-01     129.0  1037.160023  1037.160023    156.348026   
4     2022-02-01       1.0    26.230000    23.950241      6.550241   

     discount     taxes  
0    3.950000  0.234375  
1   17.100000  0.810000  
2    5.250000  0.405000  


In [8]:
print(df3.head())

               produto          categoria  \
0  2282334733936076502  Distilled Spirits   
1  6091840953834683482  Distilled Spirits   
2  1968645851245092408  Distilled Spirits   
3   994706710729219179              Draft   
4  9209550539540384349        Non-Alcohol   

                              descricao              tipos          label  \
0           JOSEPH CARTRON CAFÉ LIQUEUR  Distilled Spirits           Core   
1  SPRINGBANK 18 YEAR SINGLE MALT 700ML  Distilled Spirits      Specialty   
2     J BRANDT TRIPLE SEC 12/750ML 30PF  Distilled Spirits  Private Label   
3      REFORMATION CASHMERE IPA 1/4 KEG              Draft         In&Out   
4               HELLA MOSCOW MULE 750ML        Non Alcohol           Core   

          subcategoria                               marca  \
0  Liqueurs & Cordials                 Joseph Cartron Cafe   
1        Scotch Whisky      Springbank 18 Year Single Malt   
2  Liqueurs & Cordials                 J Brandt Triple Sec   
3          Other D

unificando as três tabelas e agregando por semana

In [9]:
df1['pdv'] = df1['pdv'].astype(str)
df2['internal_store_id'] = df2['internal_store_id'].astype(str)
df3['produto'] = df3['produto'].astype(str)
df2['internal_product_id'] = df2['internal_product_id'].astype(str)

In [10]:
df_merged = df2.merge(df1, left_on="internal_store_id", right_on="pdv", how="left")
df_merged = df_merged.merge(df3, left_on="internal_product_id", right_on="produto", how="left")

In [11]:
df_merged['reference_date'] = pd.to_datetime(df_merged['reference_date'])
df_merged['ano'] = df_merged['reference_date'].dt.year
df_merged['semana'] = df_merged['reference_date'].dt.isocalendar().week

df_semana = (
    df_merged.groupby(['ano','semana','pdv','internal_product_id',
                       'categoria','marca','fabricante','premise','categoria_pdv'])
             .agg({'quantity':'sum'})
             .reset_index()
)

In [12]:
df_semana['mes'] = pd.to_datetime(df_semana['ano'].astype(str) + df_semana['semana'].astype(str) + '1', format='%G%V%u').dt.month

In [13]:
df_semana = df_semana.sort_values(['pdv','internal_product_id','ano','semana'])

df_semana['lag1'] = df_semana.groupby(['pdv','internal_product_id'])['quantity'].shift(1)
df_semana['lag2'] = df_semana.groupby(['pdv','internal_product_id'])['quantity'].shift(2)
df_semana['lag4'] = df_semana.groupby(['pdv','internal_product_id'])['quantity'].shift(4)
df_semana['rolling_mean_4'] = df_semana.groupby(['pdv','internal_product_id'])['quantity'].shift(1).rolling(window=4).mean()

In [14]:
train = df_semana[df_semana['ano'] == 2022]
# Teste = previsões para ano=2023, semanas 1 a 5

In [15]:
# Usar semanas 49 a 52 de 2022 para calcular a média
baseline_forecast = (
    train[train['semana'] > 48]  # últimas 4 semanas
    .groupby(['pdv','internal_product_id'])['quantity']
    .mean()
    .reset_index()
)

# Renomear colunas
baseline_forecast = baseline_forecast.rename(
    columns={'internal_product_id':'produto', 'quantity':'quantidade'}
)

In [16]:
# Criar dataframe com semanas 1 a 5
horizonte = pd.DataFrame({'semana': range(1, 6)})

# Expandir previsões para todas as 5 semanas
baseline_forecast = baseline_forecast.merge(horizonte, how='cross')

# Reordenar colunas
baseline_forecast = baseline_forecast[['semana','pdv','produto','quantidade']]

In [21]:
from sklearn.metrics import mean_absolute_error

# ===============================
# 1. Dados de treino (2022)
# ===============================
train = df_semana[df_semana['ano'] == 2022]

# ===============================
# 2. Calcular baseline da média das 4 últimas semanas
# ===============================
baseline_valid = train.dropna(subset=['rolling_mean_4'])

# Previsão = rolling_mean_4
baseline_valid['forecast'] = baseline_valid['rolling_mean_4']

# ===============================
# 3. Usar apenas semanas 49–52 como validação
# ===============================
valid_real = baseline_valid[baseline_valid['semana'] > 48]

y_true = valid_real['quantity']
y_pred = valid_real['forecast']

# ===============================
# 4. Calcular métricas
# ===============================
mae = mean_absolute_error(y_true, y_pred)
wmape = (abs(y_true - y_pred).sum()) / y_true.sum()
accuracy = 1 - wmape

print(f"MAE   : {mae:.2f}")
print(f"WMAPE : {wmape:.2%}")
print(f"Accuracy: {accuracy:.2%}")

📊 Validação do Baseline (Média 4 semanas)
MAE   : 3.14
WMAPE : 49.45%
Accuracy: 50.55%


/tmp/ipython-input-1572935070.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  baseline_valid['forecast'] = baseline_valid['rolling_mean_4']


In [22]:
# Garantir que a previsão seja inteiro
baseline_forecast['quantidade'] = baseline_forecast['quantidade'].round().astype(int)

# Exportar para CSV no formato solicitado
baseline_forecast.to_csv("previsao.csv", sep=";", index=False, encoding="utf-8")